In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import mlflow
import lightgbm as lgb
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer

In [2]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Functions

In [3]:
def get_title(name):
    title=''
    for i in name:
        if i==' ':
            title=''
        elif i=='.':
            return title
        else:
            title=title+i
    return 'Without title'
def Cabin_null_processing(name):
    if (type(name) is int):
        return 0
    else:
        return 1
def Age_null_processing1(Age):
    return 37
def Age_null_processing2(Age):
    return 29
def Age_null_processing3(Age):
    return 27

## Null values processing

In [4]:
#Embarked
data['Embarked']=data['Embarked'].replace(np.nan, 'S')
test_data['Embarked']=test_data['Embarked'].replace(np.nan, 'S')

#Cabin
data['Cabin']=data['Cabin'].replace(np.nan, 0)
data['Cabin']=data['Cabin'].map(Cabin_null_processing)
data['Cabin']=data['Cabin'].replace(np.nan, 0)
data['Cabin']=data['Cabin'].map(Cabin_null_processing)
test_data['Cabin']=test_data['Cabin'].replace(np.nan, 0)
test_data['Cabin']=test_data['Cabin'].map(Cabin_null_processing)
test_data['Cabin']=test_data['Cabin'].replace(np.nan, 0)
test_data['Cabin']=test_data['Cabin'].map(Cabin_null_processing)
#data = data.drop(columns='Cabin')
#test_data = test_data.drop(columns='Cabin')
#Age
for i in  range(len(data)):
    if (pd.isna(data.loc[i, 'Age'])==True):
        if (data.loc[i,'Pclass']==1):
            data.loc[i,'Age'] = 37
        elif (data.loc[i,'Pclass']==2):
            data.loc[i,'Age'] = 29
        elif (data.loc[i,'Pclass']==3):
            data.loc[i,'Age'] = 27
for i in  range(len(test_data)):
    if (pd.isna(test_data.loc[i, 'Age'])==True):
        if (test_data.loc[i,'Pclass']==1):
            test_data.loc[i,'Age'] = 37
        elif (test_data.loc[i,'Pclass']==2):
            test_data.loc[i,'Age'] = 29
        elif (test_data.loc[i,'Pclass']==3):
            test_data.loc[i,'Age'] = 27            
#Fare
test_data['Fare']=test_data['Fare'].replace(np.nan, 20)

## Data processing

#### label encoding

In [5]:
#Sex
data['Sex']=LabelEncoder().fit_transform(data.Sex)
test_data['Sex']=LabelEncoder().fit_transform(test_data.Sex)

#Ticket
data=data.drop(columns=['Ticket'])
test_data=test_data.drop(columns=['Ticket'])
#data['Ticket']=LabelEncoder().fit_transform(data.Ticket)
#data1['Ticket']=LabelEncoder().fit_transform(data1.Ticket)

#New feature 'Title' & drop 'Name'
data['Title']=data['Name'].map(get_title)
data=data.drop(columns=['Name'])
test_data['Title']=test_data['Name'].map(get_title)
test_data=test_data.drop(columns=['Name'])

#Embarked
data['Embarked']=LabelEncoder().fit_transform(data.Embarked)
test_data['Embarked']=LabelEncoder().fit_transform(test_data.Embarked)

#PassengerId
ids = test_data['PassengerId']
#data=data.drop(columns='PassengerId')
#test_data=test_data.drop(columns='PassengerId')

#Title
data['Title']=LabelEncoder().fit_transform(data.Title)
test_data['Title']=LabelEncoder().fit_transform(test_data.Title)

In [6]:
labels = data['Survived']
data = data.drop(columns='Survived')

In [7]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,892,3,1,34.5,0,0,7.8292,0,1,5
1,893,3,0,47.0,1,0,7.0000,0,2,6
2,894,2,1,62.0,0,0,9.6875,0,1,5
3,895,3,1,27.0,0,0,8.6625,0,2,5
4,896,3,0,22.0,1,1,12.2875,0,2,6


#### one hot encoding

In [5]:
#Sex
data=pd.get_dummies(data, prefix = ['Sex'], columns = ['Sex'])
test_data=pd.get_dummies(test_data, prefix = ['Sex'], columns = ['Sex'])

In [6]:
#Ticket
data=data.drop(columns=['Ticket'])
test_data=test_data.drop(columns=['Ticket'])
#data['Ticket']=LabelEncoder().fit_transform(data.Ticket)
#data1['Ticket']=LabelEncoder().fit_transform(data1.Ticket)

In [7]:
#New feature 'Title' & drop 'Name'
data['Title']=data['Name'].map(get_title)
data=data.drop(columns=['Name'])
test_data['Title']=test_data['Name'].map(get_title)
test_data=test_data.drop(columns=['Name'])

In [8]:
#Embarked
data=pd.get_dummies(data, prefix = ['Emb'], columns = ['Embarked'])
test_data=pd.get_dummies(test_data, prefix = ['Emb'], columns = ['Embarked'])

In [9]:
ids = test_data['PassengerId']
#PassengerId
#data=data.drop(columns='PassengerId')
#data1=data1.drop(columns='PassengerId')

In [10]:
#Title
data['Title']=LabelEncoder().fit_transform(data.Title)
test_data['Title']=LabelEncoder().fit_transform(test_data.Title)

In [11]:
len(data.columns)

14

In [12]:
len(test_data.columns)

13

In [13]:
labels = data['Survived']
data = data.drop(columns='Survived')

#### Target Encoding

In [5]:
#Sex
data['Sex']=TargetEncoder().fit_transform(data.Sex, data['Survived'])
test_data['Sex']=TargetEncoder().fit_transform(test_data.Sex, data['Survived'][:418])

D:\anaconda\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
D:\anaconda\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
#Ticket
data=data.drop(columns=['Ticket'])
test_data=test_data.drop(columns=['Ticket'])
#data['Ticket']=LabelEncoder().fit_transform(data.Ticket)
#data1['Ticket']=LabelEncoder().fit_transform(data1.Ticket)

In [7]:
#New feature 'Title' & drop 'Name'
data['Title']=data['Name'].map(get_title)
data=data.drop(columns=['Name'])
test_data['Title']=test_data['Name'].map(get_title)
test_data=test_data.drop(columns=['Name'])

In [8]:
#Embarked
data['Embarked']=TargetEncoder().fit_transform(data.Embarked, data['Survived'])
test_data['Embarked']=TargetEncoder().fit_transform(test_data.Embarked, data['Survived'][:418])

D:\anaconda\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
D:\anaconda\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [9]:
#PassengerId
#data=data.drop(columns='PassengerId')
#data1=data1.drop(columns='PassengerId')
ids=test_data.PassengerId

In [10]:
#Title
data['Title']=TargetEncoder().fit_transform(data.Title, data['Survived'])
test_data['Title']=TargetEncoder().fit_transform(test_data.Title, data['Survived'][:418])

D:\anaconda\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
D:\anaconda\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [11]:
labels = data['Survived']
data = data.drop(columns='Survived')

# training (with different encodings)

In [8]:
cv_acc1 = []

# Random Forest

In [9]:
params = {'bootstrap': False,
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 600}
model = RandomForestClassifier(**params, random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_RF = model.predict(test_data)

# XGB

In [10]:
params={'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 10,
 'min_child_weight': 0.5,
 'n_estimators': 50,
 'reg_lambda': 0.1,
 'subsample': 0.8}
model = XGBClassifier(**params , random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_XGB = model.predict(test_data)

D:\anaconda\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:17:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:17:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:17:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:17:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

# Catboost

In [11]:
params ={'depth': 6, 'iterations': 20, 'learning_rate': 0.2}
model = CatBoostClassifier(**params, random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_Catboost = model.predict(test_data)

0:	learn: 0.6180833	total: 52.3ms	remaining: 995ms
1:	learn: 0.5676798	total: 52.9ms	remaining: 476ms
2:	learn: 0.5259805	total: 53.6ms	remaining: 304ms
3:	learn: 0.5064744	total: 53.9ms	remaining: 216ms
4:	learn: 0.4851243	total: 54.4ms	remaining: 163ms
5:	learn: 0.4670790	total: 55.5ms	remaining: 130ms
6:	learn: 0.4509785	total: 56.5ms	remaining: 105ms
7:	learn: 0.4377828	total: 57.5ms	remaining: 86.2ms
8:	learn: 0.4259909	total: 58.5ms	remaining: 71.5ms
9:	learn: 0.4157459	total: 59.5ms	remaining: 59.5ms
10:	learn: 0.4116442	total: 60.5ms	remaining: 49.5ms
11:	learn: 0.4051289	total: 61.5ms	remaining: 41ms
12:	learn: 0.4001645	total: 62.4ms	remaining: 33.6ms
13:	learn: 0.3950606	total: 63.4ms	remaining: 27.2ms
14:	learn: 0.3907065	total: 64.4ms	remaining: 21.5ms
15:	learn: 0.3892237	total: 65.3ms	remaining: 16.3ms
16:	learn: 0.3849990	total: 66.3ms	remaining: 11.7ms
17:	learn: 0.3825250	total: 67.2ms	remaining: 7.47ms
18:	learn: 0.3793567	total: 68.2ms	remaining: 3.59ms
19:	learn: 0

1:	learn: 0.5717697	total: 1.87ms	remaining: 16.8ms
2:	learn: 0.5374666	total: 2.31ms	remaining: 13.1ms
3:	learn: 0.5081590	total: 3.61ms	remaining: 14.4ms
4:	learn: 0.4903880	total: 4.58ms	remaining: 13.8ms
5:	learn: 0.4743879	total: 5.48ms	remaining: 12.8ms
6:	learn: 0.4592509	total: 6.53ms	remaining: 12.1ms
7:	learn: 0.4463197	total: 7.53ms	remaining: 11.3ms
8:	learn: 0.4425471	total: 7.96ms	remaining: 9.72ms
9:	learn: 0.4353579	total: 9.13ms	remaining: 9.13ms
10:	learn: 0.4286882	total: 10.2ms	remaining: 8.33ms
11:	learn: 0.4233025	total: 11.2ms	remaining: 7.46ms
12:	learn: 0.4188460	total: 12.2ms	remaining: 6.56ms
13:	learn: 0.4146156	total: 13.2ms	remaining: 5.65ms
14:	learn: 0.4090035	total: 14.2ms	remaining: 4.75ms
15:	learn: 0.4057544	total: 15.4ms	remaining: 3.86ms
16:	learn: 0.4012713	total: 16.8ms	remaining: 2.97ms
17:	learn: 0.3986761	total: 18.4ms	remaining: 2.04ms
18:	learn: 0.3947251	total: 19.4ms	remaining: 1.02ms
19:	learn: 0.3910862	total: 20.4ms	remaining: 0us
0:	le

# LightGBM

In [12]:
params = {'max_depth': 3, 'min_data_in_leaf': 100, 'num_leaves': 3}
model = LGBMClassifier(**params,  random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_LGBM = model.predict(data)


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

# SVM

In [13]:
params = {'C': 5, 'coef0': 0.001, 'degree': 3, 'gamma': 0.0006, 'kernel': 'rbf'}
model = SVC(**params,  random_state=42)
clf = make_pipeline(StandardScaler(), model)
#print('cv accuracy: ', cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
clf.fit(data, labels)
predictions_SVM = clf.predict(test_data)

# Logistic Regression

In [14]:
params={'C': 1.0, 'penalty': 'l2'}
model = LogisticRegression(**params)
clf = make_pipeline(StandardScaler(), model)
#print('cv accuracy: ', cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
clf.fit(data, labels)
predictions_LogisticRegression = model.predict(test_data)

# kNN

In [15]:
params={'n_neighbors': 13}
model = KNeighborsClassifier(**params)
clf = make_pipeline(StandardScaler(), model)
#print('cv accuracy: ', cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
clf.fit(data, labels)

predictions_kNN = clf.predict(test_data)

# Final predictions

In [19]:
prediction = [0 for i in range(len(test_data))]
print(len(prediction))
for i in range(len(test_data)):
    sum_predictions=predictions_XGB[i]+predictions_SVM[i]+predictions_kNN[i]+predictions_LogisticRegression[i]+predictions_Catboost[i]+predictions_LGBM[i]
    if sum_predictions>3:
        prediction[i]=1
    else:
        prediction[i]=0

print(prediction)

output = pd.DataFrame({'PassengerId': ids, 'Survived': prediction})
output.to_csv('my_submission.csv', index=False)

418
[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,

# Encoding table

In [24]:
cv_acc1=([0.82493134, 0.81033708, 0.81705368, 0.79575531, 0.78672909,
       0.79575531, 0.81707865])


In [26]:
cv_acc2 = [0.8249313358302123, 0.8126092384519351, 0.8237952559300874, 0.7957553058676654, 0.786729088639201,
 0.7946317103620475, 0.8170911360799001]

In [25]:
cv_acc3 = [0.826067415730337, 0.8170786516853932, 0.8114856429463172, 0.7900998751560551, 0.8193133583021224,
 0.8294132334581772, 0.8361548064918851]

In [27]:
models = ['RF', 'XGB', 'CatBoost', 'LGBM', 'SVM', 'Logistic Regression', 'kNN']
zeros = [0 for i in range(7)]
df = pd.DataFrame({'model': models, 'label': cv_acc1, 'one_hot': cv_acc2, 'target':cv_acc3})

In [28]:
df

,model,label,one_hot,target
0,RF,0.824931,0.824931,0.826067
1,XGB,0.810337,0.812609,0.817079
2,CatBoost,0.817054,0.823795,0.811486
3,LGBM,0.795755,0.795755,0.790100
4,SVM,0.786729,0.786729,0.819313
5,Logistic Regression,0.795755,0.794632,0.829413
6,kNN,0.817079,0.817091,0.836155
